In [1]:
import numpy as np
import pandas as pd
from os import listdir
import os

In [2]:
filepath = r"./voting/data"

all_data = []
for file in listdir(filepath):
    with open(os.path.join(filepath, file), "r") as f:
        parsed_lines = [line.rstrip("\n").split("\t") for line in f]
        all_rows = []
        for row in parsed_lines:
            all_rows.append(row)
        data = np.array(all_rows)
        all_data.append(data)
        
        # print(file)

all_voting_data = np.vstack(all_data)

In [3]:
AM3_CODE = "900003"

mask_yes = (all_voting_data[:, 13] == AM3_CODE) & (all_voting_data[:, 17] == "10")
mask_no  = (all_voting_data[:, 13] == AM3_CODE) & (all_voting_data[:, 17] == "20")
mask_901  = (all_voting_data[:, 13] == AM3_CODE) & (all_voting_data[:, 17] == "901")
mask_902  = (all_voting_data[:, 13] == AM3_CODE) & (all_voting_data[:, 17] == "902")
am3_yes = all_voting_data[mask_yes]
am3_no  = all_voting_data[mask_no]
am3_901 = all_voting_data[mask_901]
am3_902 = all_voting_data[mask_902]
am3_yes

array([['BAK', 'Baker', '43888', ..., '0', '10', '907'],
       ['BAK', 'Baker', '43888', ..., '0', '10', '1358'],
       ['BAK', 'Baker', '43888', ..., '0', '10', '1326'],
       ...,
       ['WAS', 'Washington', '43888', ..., '0', '10', '153'],
       ['WAS', 'Washington', '43888', ..., '0', '10', '976'],
       ['WAS', 'Washington', '43888', ..., '0', '10', '539']],
      dtype='<U80')

In [4]:
am3_merged = np.column_stack([am3_yes, am3_no[:, 18]])

import pandas as pd

cols_to_keep = [0, 1, 4, 5, 7, 8, 9, 11, 12, 18, 19]
am3_subset = am3_merged[:, cols_to_keep]

columns = [
    "CountyCode",            # 0
    "CountyName",            # 1
    "ElectionName",          # 4
    "PrecinctID",            # 5
    "RegisteredTotal",       # 7
    "RegisteredRepublican",  # 8
    "RegisteredDemocrat",    # 9
    "ContestName",           # 11
    "District",              # 12
    "YesVotes",              # 18
    "NoVotes"                # 19
]

df_am3 = pd.DataFrame(am3_subset, columns=columns)




In [5]:


mask_dad = df_am3['CountyCode'].str.upper() == 'DAD'
# append '.0' to DAD precincts
df_am3.loc[mask_dad, 'PrecinctID'] = df_am3.loc[mask_dad, 'PrecinctID'].astype(str) + '.0'
# strip leading zeros for non-DAD precincts only
df_am3.loc[~mask_dad, 'PrecinctID'] = df_am3.loc[~mask_dad, 'PrecinctID'].astype(str).str.lstrip('0')


mask_atl = df_am3['CountyCode'].astype(str).str.strip() == 'STL'
df_am3.loc[mask_atl, 'PrecinctID'] = df_am3.loc[mask_atl, 'PrecinctID'].astype(str).str.lstrip(' ')
# rebuild key after changes
df_am3['key'] = df_am3['CountyCode'] + df_am3['PrecinctID']





In [6]:
output_path = "amendment3_precinct_results.csv"
df_am3.to_csv(output_path, index=False, encoding="utf-8")
print(f"Saved to {output_path}")

Saved to amendment3_precinct_results.csv


In [7]:
mask = (df_am3['CountyCode'].str.upper() == 'STL')
df_bro_a001 = df_am3[mask]
df_bro_a001 # OKE DES MRN 


,CountyCode,CountyName,ElectionName,PrecinctID,RegisteredTotal,RegisteredRepublican,RegisteredDemocrat,ContestName,District,YesVotes,NoVotes,key
4123,STL,St. Lucie,2024 General Election,1,6054,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,2554,1817,STL1
4124,STL,St. Lucie,2024 General Election,6,3890,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,1675,1052,STL6
4125,STL,St. Lucie,2024 General Election,8,3341,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,1472,1188,STL8
4126,STL,St. Lucie,2024 General Election,9,3111,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,1335,1270,STL9
4127,STL,St. Lucie,2024 General Election,10,1941,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,790,402,STL10
...,...,...,...,...,...,...,...,...,...,...,...,...
4179,STL,St. Lucie,2024 General Election,90,2847,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,1364,1076,STL90
4180,STL,St. Lucie,2024 General Election,91,1409,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,666,458,STL91
4181,STL,St. Lucie,2024 General Election,92,10280,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,5207,3752,STL92
4182,STL,St. Lucie,2024 General Election,94,5,0,0,Amendment No. 3: Adult Personal Use of Marijuana,,1,3,STL94
